## This class will hold the database creation and population work

In [1]:
import G1_common_tools as tools

#### Install sqlalchemy_utils and Faker if they aren't already installed:

In [2]:
tools.install_required_packages()

sqlalchemy_utils is already installed.
Faker not found. Installing...
Faker has been installed.
fastapi[all] not found. Installing...
fastapi[all] has been installed.


#### Create the database:

In [3]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy_utils import database_exists, create_database

engine = create_engine('sqlite:///it_ticketing_system.db', echo=True)
if not database_exists(engine.url):
    create_database(engine.url)

if sqlalchemy.__version__ < '2.0.0':
    raise ValueError('Please upgrade your version of SQLAlchemy to 2.0.0 or greater')


### Using sqlite3 to drop tables if they're already created.

In [4]:
tools.drop_all_tables('it_ticketing_system.db')

Dropped table 'dim_organizations'
Dropped table 'dim_departments'
Dropped table 'dim_users'
Dropped table 'dim_technicians'
Dropped table 'fact_tickets'
Dropped table 'fact_ticket_lines'


#### Create the tables for the database:

In [5]:
import table_structures as tables
tables.Base.metadata.create_all(engine)

2023-10-04 18:37:09,576 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-04 18:37:09,577 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dim_organizations")
2023-10-04 18:37:09,579 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-04 18:37:09,582 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dim_organizations")
2023-10-04 18:37:09,583 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-04 18:37:09,585 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dim_departments")
2023-10-04 18:37:09,587 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-04 18:37:09,590 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dim_departments")
2023-10-04 18:37:09,591 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-04 18:37:09,593 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dim_users")
2023-10-04 18:37:09,595 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-04 18:37:09,595 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dim_users")
2023-10-04 18:37:09,595 IN

#### Initial Database Definitions:

In [6]:
tools.print_database_definitions('it_ticketing_system.db')

------------------
Table Name: dim_organizations
------------------
  Column: organization_id
    Type: INTEGER
    Not Null: YES
    Default Value: None
    Primary Key: YES
  Column: name
    Type: VARCHAR
    Not Null: YES
    Default Value: None
    Primary Key: NO
  Column: phone_number
    Type: VARCHAR
    Not Null: NO
    Default Value: None
    Primary Key: NO
  Column: email_address
    Type: VARCHAR
    Not Null: YES
    Default Value: None
    Primary Key: NO
  Column: state
    Type: VARCHAR
    Not Null: YES
    Default Value: None
    Primary Key: NO
  Column: city
    Type: VARCHAR
    Not Null: YES
    Default Value: None
    Primary Key: NO
  Column: zip_code
    Type: INTEGER
    Not Null: YES
    Default Value: None
    Primary Key: NO
  Column: street_address
    Type: VARCHAR
    Not Null: YES
    Default Value: None
    Primary Key: NO
------------------
Table Name: dim_departments
------------------
  Column: department_id
    Type: INTEGER
    Not Null: YES
   

#### Creating method to add be able to pass a List[Object] and have them added to the database, where Object is the table in ORM form

In [7]:
from sqlalchemy.orm import Session
from sqlalchemy.exc import SQLAlchemyError

def add_objects_to_db(session: Session, objects: list):
    try:
        session.add_all(objects)
        session.commit()
        return True
    except SQLAlchemyError as e:
        print(f"An error occurred: {e}")
        session.rollback()
        return False


#### Seed the tables:

In [8]:
from sqlalchemy.orm import Session
from faker import Faker
import random

fake = Faker()

def seed_organizations(session: Session, num_organizations: int):
    organizations = []
    for _ in range(num_organizations):
        organization = tables.Organization(
            name=fake.company(),
            phone_number=fake.phone_number(),
            email_address=fake.company_email(),
            state=fake.state_abbr(),
            city=fake.city(),
            zip_code=fake.zipcode(),
            street_address=fake.street_address(),
        )
        organizations.append(organization)
    return add_objects_to_db(session, organizations)

def seed_departments(session: Session, num_departments_per_organization: int, organizations: list):
    departments = []
    for organization in organizations:
        for _ in range(num_departments_per_organization):
            department = tables.Department(
                organization_id=organization.organization_id,
                name=fake.bs(),
                phone_number=fake.phone_number(),
                email_address=fake.company_email(),
            )
            departments.append(department)
    return add_objects_to_db(session, departments)

def seed_users(session: Session, num_users_per_department: int, departments: list):
    users = []
    for department in departments:
        for _ in range(num_users_per_department):
            user = tables.User(
                organization_id=department.organization_id,
                department_id=department.department_id,
                last_name=fake.last_name(),
                first_name=fake.first_name(),
                phone_number=fake.phone_number(),
                email_address=fake.email(),
                title=fake.job(),
            )
            users.append(user)
    return add_objects_to_db(session, users)

def seed_tickets(session: Session, users: list, num_tickets_per_user: int=None):
    tickets = []
    for user in users:
        user_ticket_count = 0
        if not num_tickets_per_user:
            user_ticket_count = random.randint(0, 5)
        else:
            user_ticket_count = num_tickets_per_user

        for _ in range(user_ticket_count):
            ticket = tables.Ticket(
                user_id=user.user_id,
                department_id=user.department_id,
                ticket_category=fake.word(),
                open_date_time=fake.date_time_this_decade(),
                close_date_time=fake.date_time_this_decade(),
                status=fake.word(),
                description=fake.text(),
                subject=fake.sentence(),
            )
            tickets.append(ticket)
    return add_objects_to_db(session, tickets)

def seed_technicians(session: Session, users: list, num_technicians: int):
    technicians = []
    selected_users = random.sample(users, k=min(num_technicians, len(users)))

    for user in selected_users:
        technician = tables.Technician(user_id=user.user_id)
        technicians.append(technician)

    return add_objects_to_db(session, technicians)

def seed_ticket_lines(session: Session, num_ticket_lines_per_ticket: int, tickets: list, technicians: list):
    ticket_lines = []
    for ticket in tickets:
        for _ in range(num_ticket_lines_per_ticket):
            technician = random.choice(technicians)

            ticket_line = tables.TicketLine(
                ticket_id=ticket.ticket_id,
                technician_id=technician.technician_id,
                assignment_date_time=fake.date_time_this_decade(),
                completion_date_time=fake.date_time_this_decade(),
                notes=fake.text(),
            )
            ticket_lines.append(ticket_line)
    return add_objects_to_db(session, ticket_lines)

session = Session(engine)

#Create the organizations:
num_organizations = 5
seed_organizations(session, num_organizations)
organizations = session.query(tables.Organization).all()

#Create the departments:
num_departments_per_organization = 3
seed_departments(session, num_departments_per_organization, organizations)
departments = session.query(tables.Department).all()

#Create the users:
num_users_per_department = 10
seed_users(session, 10, departments)
users = session.query(tables.User).all()

#Create the tickets:
# num_tickets_per_user = 5
# seed_tickets(session, users, num_tickets_per_user)

# Modified to randomly give each user between 0 and 5 tickets:
seed_tickets(session, users)
tickets = session.query(tables.Ticket).all()

#Create the technicians:
num_technicians = 3
seed_technicians(session, users, num_technicians)
technicians = session.query(tables.Technician).all()

#Create the ticket lines:
num_ticket_lines_per_ticket = 3
seed_ticket_lines(session, num_ticket_lines_per_ticket, tickets, technicians)
ticketlines = session.query(tables.TicketLine).all()




2023-10-04 18:37:10,195 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-04 18:37:10,195 INFO sqlalchemy.engine.Engine INSERT INTO dim_organizations (name, phone_number, email_address, state, city, zip_code, street_address) VALUES (?, ?, ?, ?, ?, ?, ?) RETURNING organization_id
2023-10-04 18:37:10,195 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/5 (ordered; batch not supported)] ('Jackson PLC', '469.736.8026', 'michaeljones@gordon-moore.net', 'SC', 'Alvaradoside', '92220', '726 Christopher Lane Suite 900')
2023-10-04 18:37:10,211 INFO sqlalchemy.engine.Engine INSERT INTO dim_organizations (name, phone_number, email_address, state, city, zip_code, street_address) VALUES (?, ?, ?, ?, ?, ?, ?) RETURNING organization_id
2023-10-04 18:37:10,211 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/5 (ordered; batch not supported)] ('Yu LLC', '669-342-1990', 'bethanyhamilton@smith.com', 'MH', 'Lake Margaretville', '22924', '36728 Desiree Plaza')
2023-10-04 18:3

In [9]:


print(f'date_time_this_decade: {fake.date_time_this_decade()}')
print(f'word: {fake.word()}')
print(f'sentence: {fake.sentence()}')
print(f'text: {fake.text()}')
print(f'first_name: {fake.first_name()}')
print(f'last_name: {fake.last_name()}')
print(f'phone_number: {fake.phone_number()}')
print(f'email: {fake.email()}')
print(f'job: {fake.job()}')
print(f'company: {fake.company()}')
print(f'company_email: {fake.company_email()}')
print(f'state_abbr: {fake.state_abbr()}')
print(f'city: {fake.city()}')
print(f'zipcode: {fake.zipcode()}')
print(f'street_address: {fake.street_address()}')
print(f'bs: {fake.bs()}')





#Tickets:
#ticket_category
#open_date_time
#close_date_time
#status
#description
#subject
#
#Ticket Lines:
#assignment_date_time
#completion_date_time
#notes
#
#Users
#last_name
#first_name
#phone_number
#email_address
#title
#
#Departments
#name
#phone_number
#
#Organizations
#name
#phone_number
#email_address
#state
#city
#zip_code
#street_address
#
#
#






date_time_this_decade: 2022-08-24 14:30:44
word: practice
sentence: Moment box quality whole wrong deal.
text: Box doctor others must example. Explain chair society chair assume base. Role call training white.
Need on keep fast brother animal entire.
first_name: Sheila
last_name: Garcia
phone_number: (385)725-9847x383
email: vanessarichards@example.org
job: Chief Financial Officer
company: Johnson Ltd
company_email: garzanicholas@hamilton.info
state_abbr: GU
city: Christinaville
zipcode: 69816
street_address: 59790 Hernandez Meadow Suite 915
bs: iterate best-of-breed partnerships


In [10]:
import pandas as pd

date_time_pre=fake.date_time_this_decade()

hours_taken = fake.random_int(min=0, max=4)
minutes_taken = fake.random_int(min=0, max=59)

date_time_post = date_time_pre + pd.DateOffset(hours=hours_taken, minutes=minutes_taken)


print(date_time_pre)
print(hours_taken)

print(minutes_taken)
print(date_time_post)


2022-01-26 15:26:26
2
32
2022-01-26 17:58:26


In [11]:
#Reset the department names:
department_names = [
    "IT Support",
    "Customer Service",
    "Marketing",
    "Finance",
    "Human Resources",
    "Sales",
    "Research and Development",
    "Legal",
    "Facilities Management",
    "Quality Assurance",
    "Product Management",
    "Supply Chain",
    "Administration",
    "Engineering",
    "Public Relations"

]

departments = session.query(tables.Department).all()

for department in departments:
    selected_department = fake.random_int(min=0, max=len(department_names)-1)
    department.name = department_names[selected_department]
session.commit()

#Reset organization names:
organization_names = [
    "Acme Corporation",
    "Tech Solutions Inc.",
    "Global Enterprises",
    "Marketing Innovators",
    "Finance World Group"
]

organizations = session.query(tables.Organization).all()

for organizations in organizations:
    selected_organization = fake.random_int(min=0, max=len(organization_names)-1)
    organizations.name = organization_names[selected_organization]
session.commit()



2023-10-04 18:37:25,760 INFO sqlalchemy.engine.Engine SELECT dim_departments.department_id AS dim_departments_department_id, dim_departments.organization_id AS dim_departments_organization_id, dim_departments.name AS dim_departments_name, dim_departments.phone_number AS dim_departments_phone_number, dim_departments.email_address AS dim_departments_email_address 
FROM dim_departments
2023-10-04 18:37:25,762 INFO sqlalchemy.engine.Engine [cached since 15.4s ago] ()
2023-10-04 18:37:25,773 INFO sqlalchemy.engine.Engine UPDATE dim_departments SET name=? WHERE dim_departments.department_id = ?
2023-10-04 18:37:25,781 INFO sqlalchemy.engine.Engine [generated in 0.00625s] [('Marketing', 1), ('Public Relations', 2), ('Sales', 3), ('Human Resources', 4), ('Supply Chain', 5), ('IT Support', 6), ('Administration', 7), ('Administration', 8)  ... displaying 10 of 15 total bound parameter sets ...  ('Public Relations', 14), ('Administration', 15)]
2023-10-04 18:37:25,790 INFO sqlalchemy.engine.Engin